In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [5]:
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [ ]:
df = pd.read_csv("./data/titanic.csv")
df.head()

In [ ]:
parser = PandasDataFrameOutputParser(dataframe=df)

print(parser.get_format_instructions())

In [ ]:
df_query = "Age column 을 조회해 주세요."

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"], 
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

chain = prompt | model | parser

parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)


In [ ]:
df_query = "Retrieve the first row."

parser_output = chain.invoke({"query": df_query})

format_parser_output(parser_output)

In [ ]:
df["Age"].head().mean()

In [ ]:
df_query = "Retrieve the average of the Ages from row 0 to 4."

parser_output = chain.invoke({"query": df_query})

print(parser_output)

In [ ]:
df_query = "Calculate average `Fare` rate."

parser_output = chain.invoke({"query": df_query})

print(parser_output)


In [ ]:
df["Fare"].mean()